In [51]:
import heapq
import csv
from datetime import datetime
from pm4py.objects.log.obj import EventLog, Trace, Event
from pm4py.objects.log.exporter.xes import exporter as xes_exporter
import random
from datetime import timedelta
from pm4py.objects.bpmn.importer import importer as bpmn_importer
from pm4py.objects.conversion.bpmn import converter as bpmn_converter


from pm4py.objects.petri_net.semantics import ClassicSemantics


class EventItem:
    def __init__(self, time, activity, case_id, data=None):
        self.time = time
        self.activity = activity
        self.case_id = case_id
        self.data = data or {}

    def __lt__(self, other):
        return self.time < other.time


class SimulationEngine:
    def __init__(self):
        self.event_queue = []
        self.event_log = []   # For CSV logging
        self.xes_log = EventLog()  # For XES logging

    # ------------------------------------------------------
    # Schedule an event
    # ------------------------------------------------------
    def schedule(self, time, activity, case_id, data=None):
        event = EventItem(time, activity, case_id, data)
        heapq.heappush(self.event_queue, event)

    # ------------------------------------------------------
    # Run the simulation
    # ------------------------------------------------------
    def run(self):
        while self.event_queue:
            event = heapq.heappop(self.event_queue)
            self._process_event(event)

    # ------------------------------------------------------
    # Process event: logging + routing
    # ------------------------------------------------------
    def _process_event(self, event):
        # CSV log
        row = {
            "time:timestamp": event.time,
            "concept:name": event.activity,
            "case:concept:name": event.case_id,
        }
        # add additional attributes
        row.update(event.data)
        self.event_log.append(row)

        # XES log
        trace = next(
            (t for t in self.xes_log
             if t.attributes.get("concept:name") == event.case_id),
            None
        )
        if trace is None:
            trace = Trace()
            trace.attributes["concept:name"] = event.case_id
            self.xes_log.append(trace)

        xes_event = Event({
            "concept:name": event.activity,
            "time:timestamp": event.time
        })
        for k, v in event.data.items():
            xes_event[k] = v
        trace.append(xes_event)

    # ------------------------------------------------------
    # Export CSV
    # ------------------------------------------------------
    def export_csv(self, filename="simulation_log.csv"):
        if not self.event_log:
            return
        keys = self.event_log[0].keys()
        with open(filename, "w", newline="") as f:
            writer = csv.DictWriter(f, fieldnames=keys)
            writer.writeheader()
            writer.writerows(self.event_log)

    def export_xes(self, filename="simulation_log.xes"):
        xes_exporter.apply(self.xes_log, filename)


class BpmnSimulationEngine(SimulationEngine):
    def __init__(self, net, im, fm, activity_durations=None):
        super().__init__()
        self.net = net
        self.im = im
        self.fm = fm

        # Use the semantics available to you
        self.semantics = ClassicSemantics()

        self.activity_durations = activity_durations or {}

    @classmethod
    def from_bpmn(cls, path, activity_durations=None):
        bpmn_graph = bpmn_importer.apply(path)
        net, im, fm = bpmn_converter.apply(bpmn_graph)
        return cls(net, im, fm, activity_durations)

    def is_final_marking(self, marking):
        """Simple final marking check since your PM4Py version does not provide one."""
        for place in self.fm:
            if marking.get(place, 0) != self.fm[place]:
                return False
        return True

    from datetime import timedelta
    import random

    def simulate_case(self, case_id, start_time):
        marking = self.im.copy()
        current_time = start_time
    
        MAX_STEPS = 50000
        steps = 0
    
        # Speicher für Marking-Wiederholungen
        last_markings = {}
    
        while steps < MAX_STEPS:
            steps += 1
    
            # 1) Final marking erreicht?
            if self.is_final_marking(marking):
                break
    
            # 2) Enabled transitions
            enabled = self.semantics.enabled_transitions(self.net, marking)
            if not enabled:
                print(f"[WARN] Deadlock in case {case_id}: no transitions enabled.")
                break
    
            invisible = [t for t in enabled if t.label is None]
            visible = [t for t in enabled if t.label is not None]
    
            # 3) Marking beobachten (Schlüssel für Wiederholungen)
            marking_key = tuple(sorted((p.name, c) for p, c in marking.items()))
            last_markings[marking_key] = last_markings.get(marking_key, 0) + 1
    
            # 🚨 Wenn Marking zu oft identisch bleibt → Endlosschleife
            if last_markings[marking_key] > 200:
                print(f"[ERROR] Infinite structural loop detected in case {case_id}. "
                      f"Marking={marking}")
                break
    
            # 4) Unsichtbare Transitionen (τ) haben Vorrang
            if invisible:
                t = random.choice(invisible)
                marking = self.semantics.execute(t, self.net, marking)
                continue
    
            # 5) Sichtbare Transitionen = echte Aktivitäten
            t = random.choice(visible)
            activity = t.label
    
            # Dauer bestimmen (Default 1h)
            duration = self.activity_durations.get(activity, timedelta(hours=1))
    
            start_ts = current_time
            end_ts = current_time + duration
    
            # Lifecycle Events loggen
            self.schedule(start_ts, activity, case_id,
                          data={"lifecycle:transition": "start"})
            self.schedule(end_ts, activity, case_id,
                          data={"lifecycle:transition": "complete"})
    
            current_time = end_ts
    
            # Transition feuern
            marking = self.semantics.execute(t, self.net, marking)
    
        else:
            print(f"[ERROR] MAX_STEPS reached for case {case_id} – model may not converge.")
    
    
            
    
    
activity_durations = {
    "A_Create Application": timedelta(hours=1),
    "A_Submitted": timedelta(hours=2),
    "A_Concept": timedelta(hours=3),
    "A_Accepted": timedelta(hours=1),
    "O_Create Offer": timedelta(hours=4),
}

engine = BpmnSimulationEngine.from_bpmn(
    "SimonsTryFürCoreEngine:(/BPMN.bpmn",
    activity_durations=activity_durations
)

start = datetime.now()

engine.simulate_case("C1", start)
engine.simulate_case("C2", start + timedelta(minutes=5))

engine.run()
engine.export_csv("sim_bpmn.csv")
engine.export_xes("sim_bpmn.xes")


exporting log, completed traces ::   0%|          | 0/2 [00:00<?, ?it/s]